In [2]:
!pip install -q condacolab
import condacolab
condacolab.install()
!conda create --name shap-e python=3.10 -y

!conda activate shap-e
!conda install pytorch torchvision torchaudio pytorch-cuda=11.8 pytorch3d jupyter git -c pytorch -c nvidia -c pytorch3d -y
!git clone https://github.com/openai/shap-e.git
%cd shap-e
!pip install cchardet chardet pyyaml
!pip install -e .



✨🍰✨ Everything looks OK!
Solving environment: - \ done


==> WARNING: A newer version of conda exists. <==
  current version: 23.1.0
  latest version: 23.11.0

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.11.0



## Package Plan ##

  environment location: /usr/local/envs/shap-e

  added / updated specs:
    - python=3.10


The following NEW packages will be INSTALLED:

  _libgcc_mutex      conda-forge/linux-64::_libgcc_mutex-0.1-conda_forge 
  _openmp_mutex      conda-forge/linux-64::_openmp_mutex-4.5-2_gnu 
  bzip2              conda-forge/linux-64::bzip2-1.0.8-hd590300_5 
  ca-certificates    conda-forge/linux-64::ca-certificates-2023.11.17-hbcca054_0 
  ld_impl_linux-64   conda-forge/linux-64::ld_impl_linux-64-2.40-h41732ed_0 
  libffi             conda-forge/linux-64::libffi-3.4.2-h7f98852_5 
  libgcc-ng          conda-forge/linux-64::libgcc-ng

In [3]:
import torch

from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

xm = load_model('transmitter', device=device)
model = load_model('text300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))



  0%|          | 0.00/1.78G [00:00<?, ?iB/s]

100%|███████████████████████████████████████| 890M/890M [00:11<00:00, 84.5MiB/s]


  0%|          | 0.00/1.26G [00:00<?, ?iB/s]

In [4]:
batch_size = 4
guidance_scale = 15.0
prompt = "A Lion"

latents = sample_latents(
    batch_size=batch_size,
    model=model,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(texts=[prompt] * batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=64,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
)

  0%|          | 0/64 [00:00<?, ?it/s]

In [5]:
render_mode = 'nerf' # you can change this to 'stf'
size = 64 # this is the size of the renders; higher values take longer to render.

cameras = create_pan_cameras(size, device)
for i, latent in enumerate(latents):
    images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
    display(gif_widget(images))

# Example of saving the latents as meshes.
from shap_e.util.notebooks import decode_latent_mesh

for i, latent in enumerate(latents):
    t = decode_latent_mesh(xm, latent).tri_mesh()
    with open(f'example_mesh_{i}.ply', 'wb') as f:
        t.write_ply(f)
    with open(f'example_mesh_{i}.obj', 'w') as f:
        t.write_obj(f)



HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAANfOu9XMu9PKtdDFr87Crcm+qcu5nce2l8G0m8OxlMCwkcKvj8…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAANDCos7Bo87Bocq7nca4mcS4mMO4lcm3lse3l8a2l8W2k8W1j8…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAALGzsa+wrq2vra2urayurayurK2trKytrKytq6utrKutq6utqq…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAALSzsbOzsbKxr7GwrrGwrbCvrLCurK6trK6tqq6sqa2rqKyqp6…

/content/shap-e/shap_e/models/stf/renderer.py:286: UserWarning: exception rendering with PyTorch3D: No module named 'pytorch3d'
  warnings.warn(f"exception rendering with PyTorch3D: {exc}")
/content/shap-e/shap_e/models/stf/renderer.py:287: UserWarning: falling back on native PyTorch renderer, which does not support full gradients
  warnings.warn(


In [6]:
batch_size = 1
guidance_scale = 15.0
prompt = "A Lion"

latents = sample_latents(
    batch_size=batch_size,
    model=model,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(texts=[prompt] * batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=64,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
)

  0%|          | 0/64 [00:00<?, ?it/s]

In [7]:
render_mode = 'nerf' # you can change this to 'stf'
size = 256 # this is the size of the renders; higher values take longer to render.

cameras = create_pan_cameras(size, device)
for i, latent in enumerate(latents):
    images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
    display(gif_widget(images))

# Example of saving the latents as meshes.
from shap_e.util.notebooks import decode_latent_mesh

for i, latent in enumerate(latents):
    t = decode_latent_mesh(xm, latent).tri_mesh()
    with open(f'example_mesh_{i}.ply', 'wb') as f:
        t.write_ply(f)
    with open(f'example_mesh_{i}.obj', 'w') as f:
        t.write_obj(f)



HTML(value='<img src="data:image/gif;base64,R0lGODlhAAEAAYcAAL6gebyedryddb2cdbucdLybc7uacruZcbiZcraYdLmXb7eVbr…